- grafo cujos nodes são os produtos filtrados na interface por categoria simplificada e rede de fast_food
- tamanho dos nodes é proporcional a % de nutrientes nos produtos
- tamanho das arestas é proporcional a % de nutrientes comuns entre os 2 produtos

In [3]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import pathlib
import getpass

# !pip install pymysql
import pymysql

In [32]:
p = getpass.getpass()
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd = p) 
cur = conn.cursor()

 ·········


In [33]:
cur.execute("USE fast_food;")

0

In [4]:
cur.execute("SHOW tables;")
for r in cur.fetchall():
   print(r)

('address',)
('category',)
('chain',)
('nutricional_units',)
('product',)
('restaurant',)


In [26]:
# simplificar categorias
cur.execute("SELECT * FROM category;") 
for r in cur.fetchall():
   print(r)

(1, 'WHOPPER')
(2, 'MEAT SANDWICH')
(3, 'CHICKEN SANDWICH')
(4, 'VEGGIES')
(5, 'SIDE DISH')
(6, 'SAUCE')
(7, 'ICE CREAM')
(8, 'SHAKES')
(9, 'BREAKFAST')
(10, 'BEEF & PORK')
(11, 'CHICKEN & FISH')
(12, 'SALADS')
(13, 'SNACKS & SIDES')
(14, 'DESSERTS')
(15, 'BEVERAGES')
(16, 'COFFEE & TEA')
(17, 'SMOOTHIES & SHAKES')
(18, 'CROISSANT')
(19, 'HEALTHY BREAD')
(20, 'BAR AND SHORTBREAD')
(21, 'COOKIE')
(22, 'MUFFIN')
(23, 'FAIRTRADE BANANA')
(24, 'OATMEAL')
(25, 'BREAKFAST SANDWICHES & SAVOURY MUFFINS')
(26, 'TOASTIES')
(27, 'SANDWICHES, PANINIS, WRAPS, ROLLS & FOCACCIAS')
(28, 'HOT BOXES')
(29, 'SALAD BOWLS')
(30, 'COFFEE')
(31, 'CAPPUCCINO')
(32, 'MACCHIATO')
(33, ' DOLCE LATTE')
(34, 'MOCHA')
(35, ' MOCHA')
(36, 'DOUBLESHOT ')
(37, 'TEA')
(38, 'CHAI LATTE')
(39, 'REFRESHERS')
(40, 'SMOOTHIE')
(41, ' BLENDED BEVERAGE')
(42, 'OTHERS BEVERAGE')


In [17]:
cur.execute("SELECT * FROM chain;") 
for r in cur.fetchall():
   print(r)

(1, 'BURGER KING')
(2, 'MCDONALDS')
(3, 'STARBUCKS')


In [42]:
cur.execute("SELECT * FROM product LIMIT 5;")
for r in cur.fetchall():
   print(r)

(1, 'WHOPPER', 1, 1, 1, 361)
(2, 'WHOPPER DUPLO', 1, 2, 1, 456)
(3, 'WHOPPER FURIOSO', 1, 3, 1, 386)
(4, 'WHOPPER JR.', 1, 4, 1, 196)
(5, 'WHOPPER JR. BARBECUE BACON', 1, 5, 1, 249)


In [48]:
cur.execute("DESCRIBE product;")
for r in cur.fetchall():
   print(r)

('id_product', 'int', 'NO', 'PRI', None, 'auto_increment')
('name', 'varchar(500)', 'YES', '', None, '')
('chain_id_chain', 'int', 'NO', 'MUL', None, '')
('nutricional_units_id_nutricional_units', 'int', 'NO', 'MUL', None, '')
('category_id_category', 'int', 'NO', 'MUL', None, '')
('serving_size', 'int', 'YES', '', None, '')


In [41]:
query = '''
SELECT COUNT( product.name ) 
FROM product , category
WHERE NOT category.name = 'FAIRTRADE BANANA' ;
'''
cur.execute(query)
for r in cur.fetchall():
   print(r)

(31119,)


In [46]:
query = '''
SELECT p.name AS "product" , p.chain_id_chain AS "chain" , ca.name AS "category" , p.serving_size , n.* 
FROM  nutricional_units n 
INNER JOIN product p ON n.id_nutricional_units = p.nutricional_units_id_nutricional_units
INNER JOIN category ca ON p.category_id_category = ca.id_category
WHERE (NOT ca.name = 'FAIRTRADE BANANA');
'''
df = pd.read_sql(query, con=conn, index_col = "product")
df = df.drop(["id_nutricional_units"],axis=1)

In [47]:
df

,chain,category,serving_size,calories,total_fat,saturated_fat,trans_fat,cholesterol,sodium,carbohydrates,dietary_fiber,sugars,protein,vitamin_a,vitamin_c,calcium,iron,salt,caffeine
product,,,,,,,,,,,,,,,,,,,
WHOPPER,1,WHOPPER,361,677,35,19,0.0,NaN,1,54,2,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN
WHOPPER DUPLO,1,WHOPPER,456,976,58,27,0.0,NaN,1,50,3,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN
WHOPPER FURIOSO,1,WHOPPER,386,838,42,18,0.0,NaN,2,59,2,NaN,39.0,NaN,NaN,NaN,NaN,NaN,NaN
WHOPPER JR.,1,WHOPPER,196,369,18,1,0.0,NaN,631,32,2,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN
WHOPPER JR. BARBECUE BACON,1,WHOPPER,249,447,22,10,0.0,NaN,867,37,2,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWEETENED WHIPPED CREAM SHORT - HOT,3,OTHERS BEVERAGE,237,50,5,3,0.0,20.0,5,1,0,1.0,0.0,6.0,0.0,0.0,0.0,NaN,0.0
SWEETENED WHIPPED CREAM TALL - HOT,3,OTHERS BEVERAGE,355,60,6,4,0.0,20.0,5,2,0,1.0,0.0,6.0,0.0,0.0,0.0,NaN,0.0
SWEETENED WHIPPED CREAM GRANDE/VENTI® - HOT,3,OTHERS BEVERAGE,473,70,7,5,0.0,25.0,5,2,0,2.0,0.0,8.0,0.0,2.0,0.0,NaN,0.0
